In [ ]:
import pickle
from pathlib import Path

import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import config_new as config
from cardiac_electrophysiology.ulac import construction_new as construction
from cardiac_electrophysiology.ulac import workflow

In [ ]:
patient_id = "07"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = {"MV": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [ ]:
ulac_construction_settings = construction.ULACConstructorSettings(
    mesh=mesh,
    feature_tags=anatomical_tags,
    path_config=config.path_configs,
    marker_config=config.marker_configs,
    submesh_config=config.submesh_configs,
    segmentation_workflow=workflow.workflow,
)
ulac_constructor = construction.ULACConstructor(ulac_construction_settings)
ulac_constructor.construct_segmentation()
ulac_constructor.construct_uacs()

In [ ]:
marker_data = ulac_constructor.data.marker_data
uac_path_data = ulac_constructor.data.uac_path_data
marker_key_sequences = list(construction.nested_dict_keys(marker_data))
path_key_sequences = list(construction.nested_dict_keys(uac_path_data))

plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 500])
for i, uac in enumerate(["alpha", "beta"]):
    plotter.subplot(0, i)
    plotter.add_mesh(mesh, style="wireframe", color="grey")

    for key_sequence in marker_key_sequences:
        marker = construction.get_dict_entry(key_sequence, marker_data)
        if marker is None:
            continue
        marker_mesh = pv.PolyData(mesh.points[marker.ind])
        plotter.add_mesh(
            marker_mesh, color="red", point_size=10, render_points_as_spheres=True
        )
    for key_sequence in path_key_sequences:
        path = construction.get_dict_entry(key_sequence, uac_path_data)
        if path is None:
            continue
        path_mesh = pv.PolyData(mesh.points[path.inds])
        path_mesh.point_data[uac] = getattr(path, uac)
        plotter.add_mesh(path_mesh, point_size=5, scalars=uac, render_points_as_spheres=True)

plotter.show()

In [ ]:
roof_boundary = ulac_constructor.data.submesh_boundary_data["roof"]

plotter = pv.Plotter(window_size=[700, 500])
plotter.add_mesh(mesh, style="wireframe", color="grey")
roof_boundary_mesh = pv.PolyData(mesh.points[roof_boundary.inds])
roof_boundary_mesh.point_data["alpha"] = roof_boundary.alpha
roof_boundary_mesh.point_data["beta"] = roof_boundary.beta
plotter.add_mesh(
    roof_boundary_mesh, point_size=5, scalars="beta", render_points_as_spheres=True
)
plotter.show()

In [ ]:
mesh_points_without_boundary = np.setdiff1d(
    np.arange(mesh.number_of_points), roof_boundary.inds
)
mesh_without_boundary = mesh.extract_points(mesh_points_without_boundary, adjacent_cells=False)
splitted_meshes = mesh_without_boundary.split_bodies()
np.where(splitted_meshes[1].point_data["vtkOriginalPointIds"] == outside_inds[0])[0]
roof_mesh_inds = splitted_meshes[1].point_data["vtkOriginalPointIds"]
outside_inds = ulac_constructor.data.raw_path_data["MV"]
seed_ind = roof_mesh_inds[0]